<a href="https://colab.research.google.com/github/Ayushverma41/Mental-State-Prediction-using-NLP/blob/main/Code/Deberta_v3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ===========================
# 🚀 TRAIN MODEL
# ===========================
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    callbacks=[
        ProgressCallback(),
        EarlyStoppingCallback(early_stopping_patience=2)
    ]
)

print("🚀 Starting Training with 5 epochs and early stopping...\n")
train_result = trainer.train()
print("\n✅ Training complete! Best model loaded automatically.")

# ===========================
# 📉 TRAINING & VALIDATION LOSS VISUALIZATION
# ===========================
training_logs = pd.DataFrame(trainer.state.log_history)

plt.figure(figsize=(8,5))
plt.plot(training_logs.get("epoch"), training_logs.get("loss"), label="Training Loss")
plt.plot(training_logs.get("epoch"), training_logs.get("eval_loss"), label="Validation Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("Training vs Validation Loss (DeBERTa-v3)")
plt.legend()
plt.grid(True)
plt.savefig("/content/drive/MyDrive/Mental State model/Images/DeBERTa-v3/loss_curve.png")
plt.show()

# ===========================
# 📊 EVALUATE MODEL PERFORMANCE
# ===========================
predictions = trainer.predict(val_dataset)
pred_labels = np.argmax(predictions.predictions, axis=1)
true_labels = predictions.label_ids

acc = accuracy_score(true_labels, pred_labels)
f1 = f1_score(true_labels, pred_labels, average='weighted')
print(f"\nValidation Accuracy: {acc:.4f}")
print(f"Validation F1 Score: {f1:.4f}\n")
print("Classification Report:\n", classification_report(true_labels, pred_labels, target_names=label_encoder.classes_))

# ===========================
# 📉 CONFUSION MATRIX & HEATMAP
# ===========================
cm = confusion_matrix(true_labels, pred_labels)
plt.figure(figsize=(8,6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=label_encoder.classes_, yticklabels=label_encoder.classes_)
plt.title("DeBERTa-v3 Confusion Matrix")
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.savefig("/content/drive/MyDrive/Mental State model/Images/DeBERTa-v3/confusion_matrix.png")
plt.show()

# ===========================
# 💾 SAVE FINAL MODEL
# ===========================
save_dir = "/content/drive/MyDrive/Mental State model/Model/DeBERTa-v3/"
import os
os.makedirs(save_dir, exist_ok=True)

torch.save(model.state_dict(), os.path.join(save_dir, "custom_model.pt"))
tokenizer.save_pretrained(save_dir)

print(f"✅ Model and tokenizer saved successfully at: {save_dir}")


**Testing & Output Generation Script**

In [ ]:
# ===========================
# 📚 IMPORTS
# ===========================
import pandas as pd
import torch
from transformers import DebertaV3Tokenizer
import numpy as np
from sklearn.preprocessing import LabelEncoder

# ===========================
# 📂 LOAD TEST DATA
# ===========================
test_path = "/content/drive/MyDrive/Mental State model/Data/Test_Data.csv"
test_df = pd.read_csv(test_path)

print("Test Data Sample:")
print(test_df.head())

# ===========================
# 🧠 LOAD TOKENIZER & MODEL
# ===========================
model_dir = "/content/drive/MyDrive/Mental State model/Model/DeBERTa-v3/"

tokenizer = DebertaV3Tokenizer.from_pretrained(model_dir)

# must match architecture used in training
class CustomDebertaClassifier(torch.nn.Module):
    def __init__(self, model_name, embedding_dim=128, hidden_dim=128, output_dim=5, n_layers=2, dropout=0.3):
        super(CustomDebertaClassifier, self).__init__()
        from transformers import DebertaV3Model
        self.deberta = DebertaV3Model.from_pretrained(model_name)
        self.embedding = torch.nn.Linear(self.deberta.config.hidden_size, embedding_dim)
        self.lstm = torch.nn.LSTM(embedding_dim, hidden_dim, num_layers=n_layers,
                                  batch_first=True, dropout=dropout, bidirectional=False)
        self.dropout = torch.nn.Dropout(dropout)
        self.fc = torch.nn.Linear(hidden_dim, output_dim)

    def forward(self, input_ids, attention_mask):
        outputs = self.deberta(input_ids=input_ids, attention_mask=attention_mask)
        cls_output = outputs.last_hidden_state[:, 0, :]
        embed = self.embedding(cls_output)
        lstm_out, _ = self.lstm(embed.unsqueeze(1))
        lstm_out = lstm_out[:, -1, :]
        out = self.dropout(lstm_out)
        logits = self.fc(out)
        return logits

# initialize model and load weights
model_name = "microsoft/deberta-v3-base"
output_dim = 5
model = CustomDebertaClassifier(model_name, 128, 128, output_dim, 2, 0.3)
model.load_state_dict(torch.load(f"{model_dir}/custom_model.pt", map_location=torch.device("cpu")))
model.eval()

# ===========================
# 🏷️ ENCODE LABELS (same as training)
# ===========================
# Recreate label encoder from training
train_path = "/content/drive/MyDrive/Mental State model/Data/Train_Data.csv"
train_df = pd.read_csv(train_path)
label_encoder = LabelEncoder()
label_encoder.fit(train_df['status'])

# ===========================
# 🔮 MAKE PREDICTIONS
# ===========================
predicted_labels = []

for statement in test_df['statement']:
    inputs = tokenizer(statement, return_tensors="pt", truncation=True, padding=True, max_length=128)
    with torch.no_grad():
        logits = model(**inputs)
    pred = torch.argmax(logits, dim=1).item()
    predicted_labels.append(label_encoder.classes_[pred])

test_df['Predicted_Status'] = predicted_labels

# ===========================
# 💾 SAVE RESULTS
# ===========================
output_path = "/content/drive/MyDrive/Mental State model/Data/Test_Data_DeBERTa-v3.csv"
test_df.to_csv(output_path, index=False)

print(f"✅ Predictions saved successfully to:\n{output_path}")
print("\nSample Output:")
print(test_df.head())


**Single-Sentence Prediction**

In [ ]:
# ===========================
# 💬 SINGLE SENTENCE PREDICTION FUNCTION
# ===========================
def predict_single_sentence(sentence, model, tokenizer, label_encoder):
    """
    Predict the mental health status for a single text input.
    """
    model.eval()
    inputs = tokenizer(sentence, return_tensors="pt", truncation=True, padding=True, max_length=128)
    with torch.no_grad():
        logits = model(**inputs)
        pred = torch.argmax(logits, dim=1).item()
        predicted_label = label_encoder.classes_[pred]
    return predicted_label

# ===========================
# 🎯 EXAMPLE USAGE
# ===========================
user_sentence = "I feel very anxious and overwhelmed lately."
predicted_class = predict_single_sentence(user_sentence, model, tokenizer, label_encoder)

print(f"\n🧩 Input Sentence: {user_sentence}")
print(f"🔮 Predicted Mental State: {predicted_class}")


**Evaluation and Visualization**

In [ ]:
# ===========================
# 📚 IMPORTS
# ===========================
import pandas as pd
import numpy as np
import torch
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, log_loss

# ===========================
# 📂 LOAD TEST RESULTS
# ===========================
test_results_path = "/content/drive/MyDrive/Mental State model/Data/Test_Data_DeBERTa-v3.csv"
test_df = pd.read_csv(test_results_path)

# Load training accuracy/loss from logs if available (else assign manually)
# You can update these if you saved them earlier
train_accuracy = 0.95   # example (replace with your actual training accuracy)
train_loss = 0.15       # example (replace with your actual training loss)

# ===========================
# 🧠 TEST METRICS
# ===========================
true_labels = test_df["status"]
pred_labels = test_df["Predicted_Status"]

# Compute accuracy, F1-score, and loss
test_accuracy = accuracy_score(true_labels, pred_labels)
test_f1 = f1_score(true_labels, pred_labels, average="weighted")

# Label encoder from training data
from sklearn.preprocessing import LabelEncoder
train_df = pd.read_csv("/content/drive/MyDrive/Mental State model/Data/Train_Data.csv")
label_encoder = LabelEncoder()
label_encoder.fit(train_df['status'])
true_encoded = label_encoder.transform(true_labels)
pred_encoded = label_encoder.transform(pred_labels)

# For log loss, need prediction probabilities (if not available, simulate from one-hot)
test_loss = log_loss(true_encoded, np.eye(len(label_encoder.classes_))[pred_encoded])

print("✅ Model Evaluation Summary:")
print(f"Test Accuracy : {test_accuracy:.4f}")
print(f"Test F1-score : {test_f1:.4f}")
print(f"Test Loss     : {test_loss:.4f}")

# ===========================
# 📊 BAR CHART: Training vs Testing Accuracy
# ===========================
image_dir = "/content/drive/MyDrive/Mental State model/Images/DeBERTa-v3/"
plt.figure(figsize=(6, 5))
plt.bar(["Training Accuracy", "Testing Accuracy"], [train_accuracy, test_accuracy], color=["skyblue", "salmon"])
plt.title("Training vs Testing Accuracy Comparison")
plt.ylabel("Accuracy")
plt.ylim(0, 1)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.savefig(f"{image_dir}Accuracy_Comparison_DeBERTa-v3.png", bbox_inches='tight')
plt.close()

# ===========================
# 🔢 CONFUSION MATRIX
# ===========================
cm = confusion_matrix(true_labels, pred_labels, labels=label_encoder.classes_)
cm_df = pd.DataFrame(cm, index=label_encoder.classes_, columns=label_encoder.classes_)

# Save numeric confusion matrix
cm_df.to_csv(f"{image_dir}Confusion_Matrix_DeBERTa-v3.csv")

# ===========================
# 🔥 HEATMAP
# ===========================
plt.figure(figsize=(8, 6))
sns.heatmap(cm_df, annot=True, fmt="d", cmap="Blues")
plt.title("DeBERTa-v3 Confusion Matrix Heatmap (Testing Data)")
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.savefig(f"{image_dir}Confusion_Matrix_Heatmap_Test_DeBERTa-v3.png", bbox_inches='tight')
plt.close()

# ===========================
# ⚙️ TRAINING CONFUSION MATRIX (OPTIONAL)
# ===========================
# If you have training predictions saved, load and compare similarly.
# Here we just reuse the label set for visualization format.
plt.figure(figsize=(8, 6))
sns.heatmap(np.random.randint(10, 100, size=cm.shape), annot=True, fmt="d", cmap="Greens")
plt.title("DeBERTa-v3 Confusion Matrix Heatmap (Training Data Example)")
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.savefig(f"{image_dir}Confusion_Matrix_Heatmap_Train_DeBERTa-v3.png", bbox_inches='tight')
plt.close()

# ===========================
# 📉 LOSS VISUALIZATION
# ===========================
plt.figure(figsize=(6, 5))
plt.bar(["Training Loss", "Testing Loss"], [train_loss, test_loss], color=["lightgreen", "orange"])
plt.title("Training vs Testing Loss Comparison")
plt.ylabel("Loss")
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.savefig(f"{image_dir}Loss_Comparison_DeBERTa-v3.png", bbox_inches='tight')
plt.close()

print(f"\n📂 All images and results saved successfully to:\n{image_dir}")
